# Named Entity Recognition (NER)

Como hemos visto, un NER se encarga de etiquetar palabras en un texto en un conjunto de categorías (nombres de persona, de localización, de entidades de tiempo...). 

<img src=https://miro.medium.com/max/1400/1*8LOMipM-fmszClg-AwATkQ.png width=600px>

A diferencia de un problema de clasificación de documentos (en el que todo el documento es clasificado en una categoría), aquí cada palabra (o, más correctamente, cada token) es etiquetado secuencialmente.

Existen distintas aproximaciones para abordar este tipo de problemas:
- **_Clásicas_**: principalmente basados en reglas
- **Machine Learning**: se distinguen a su vez dos categorías aquí:
    - **Clasificación multi-clase**: se busca clasificar cada token entre un conjunto de categorías sin información del contexto del token
    - **CRF**: modelar el texto secuencialmente mediante un grafo probabilístico. En cada token se extraen features que tratan de representar su contexto. [Aquí](https://www.depends-on-the-definition.com/named-entity-recognition-conditional-random-fields-python/) lo explican muy bien.
- **Deep learning**: estado del arte (como en casi todas las áreas). Utilizar modelos como las Bi-LSTM permite inferir el contexto de un token de una manera más completa tanto de tokens anteriores a el, como posteriores.

**Nosotros vamos a implementar una solución basada en un CRF**.

<img src=https://i.imgur.com/ukAr3Uh.jpg width=700px>


## Pasos que seguiremos para entrenar un modelo de reconocimiento de entidades

Etapa de **entrenamiento**
1. Obtener un corpus de entrenamiento
2. Etiquetar todos los tokens. Aquellos que no correspondan con una categoría se les asocia el label 'O'
3. Definir qué features se extraerán 
4. Entrenar un modelo de clasificación secuencial para predecir las etiquetas de los tokens

Etapa de **testeo**
1. Obtener un corpus de test etiquetado
2. Lanzar el modelo etiquetando los tokens
3. Analizar resultados


## IO / IOB Encoding

Existen dos maneras principales de etiquetas las etiquetas (IO e IOB).

- **IO** (inside-ouside): cada token tendrá una etiqueta, **no tiene en cuenta entidades compuestas por varios tokens**, o chunks, como, por ejemplo, 'Nueva York'.
- **IOB** (inside-outside-beginning): cada token tendrá una etiqueta. Si tiene en cuenta chunks. Para codificar el inicio y final de las entidades se incluyen los prefijos 'B-' ('Beginning') e 'I-' ('Inside') para indicar que un token es el inicio de una entidad o pertenece a la misma.

En ambos, la etiqueta **'O'** ('Outside') significa que el token no pertenece a ninguna entidad.

<img src=https://image.slidesharecdn.com/07lectiener-160215132149/95/ie-named-entity-recognition-ner-36-638.jpg>

> Debate: ¿cuál pensáis que arroja mejores resultados?


## Features

Algunas de las features que podemos extraer son:
- **Palabras**
    - El token actual e información sobre el mismo (mayúsculas, signos de puntuación, ...)
    - Tokens anteriores / posteriores e información sobre ellos
    - Substrings (word shapes)
- **Información lingüística**
    - PoS tags (del token, de los anteriores / consecutivos)
- **Otras labels**
    - NER labels (del token actual, de los anteriores)


## ¡Al lío!

Vamos a entrenar nuestro primer modelos de reconocimiento de entidades. Para ello, utilizaremos:
- Dataset CoNLL 2002, con PoS Tags
- Librería sklearn_crf suite

# CoNLL 2002 Shared Tasks con PoS Tags

La CoNLL (Conference on Computational Natural Language Learning) es una conferencia anual organizada por la SIGNLL (ACL's Special Interest Group of Natural Language Processing). Usaremos un corpus con **frases en castellano** con información tanto de los **PoS Tags** como de **entidades etiquetadas**.

Links:
- https://www.cs.upc.edu/~nlp/tools/nerc/nerc.html
- https://www.plantl.gob.es/tecnologias-lenguaje/catalogo-TL/campanas-evaluacion/Paginas/conll-2002.aspx

# sklearn_crfsuite

Instalación: `pip install sklearn-crfsuite`

Links:
- Tutorial: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html
- API reference: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

In [ ]:
# !pip3 install sklearn_crfsuite

In [1]:
import os 
import io

import pandas as pd

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# Funciones útiles

In [2]:
def load_data(filepath):
    with io.open(filepath, encoding='latin-1') as f:
        idsent = 0
        sentences = []
        for l in f:
            if not len(l.strip()):
                sentence = []
                idsent += 1
            else:
                word, pos, ner = l.strip().split(' ')
                sentences.append(['sentence#'+str(idsent), word, pos, ner])
                
        df = pd.DataFrame(sentences)
        df.columns = ['Sentence#','word','pos','ner']
        
        return df

In [3]:
def data_summary(data):
    print("Number of sentences: ", len(data.groupby(['Sentence#'])))

    words = list(set(data["word"].values))
    n_words = len(words)
    print("Number of words in the dataset: ", n_words)

    tags = list(set(data["ner"].values))
    print("NER Tags:", tags)
    n_tags = len(tags)
    print("Number of Labels: ", n_tags)

    print("What the dataset looks like:")
    display(data.head(10))
    return

In [4]:
def get_sentences(data):
    sentgroups = data.groupby('Sentence#')

    sentences = []
    for name, g in sentgroups:
        s = []
        for row in g.itertuples():
            s.append((row.word,row.pos,row.ner))
        sentences.append(s)
    
    return sentences

In [5]:
def get_labels(data):
    tags = list(set(data["ner"].values))
    return tags

# Carga de datos

In [6]:
datasets_path = '../../datasets/CoNLL2002_NER'
corpus_train_file = 'esp.train'
corpus_test_file = 'esp.testa'
corpus_val_file = 'esp.testb'

In [7]:
df_train = load_data(os.path.join(datasets_path, corpus_train_file))
df_test = load_data(os.path.join(datasets_path, corpus_test_file))
df_val = load_data(os.path.join(datasets_path, corpus_val_file))

In [8]:
sentences_train = get_sentences(df_train)
sentences_test = get_sentences(df_test)
sentences_val = get_sentences(df_val)

In [9]:
data_summary(df_train)

Number of sentences:  8323
Number of words in the dataset:  26099
NER Tags: ['B-ORG', 'I-ORG', 'I-LOC', 'I-MISC', 'I-PER', 'B-PER', 'B-LOC', 'O', 'B-MISC']
Number of Labels:  9
What the dataset looks like:


,Sentence#,word,pos,ner
0,sentence#0,Melbourne,NP,B-LOC
1,sentence#0,(,Fpa,O
2,sentence#0,Australia,NP,B-LOC
3,sentence#0,),Fpt,O
4,sentence#0,",",Fc,O
5,sentence#0,25,Z,O
6,sentence#0,may,NC,O
7,sentence#0,(,Fpa,O
8,sentence#0,EFE,NC,B-ORG
9,sentence#0,),Fpt,O


In [10]:
data_summary(df_test)

Number of sentences:  1915
Number of words in the dataset:  9646
NER Tags: ['B-ORG', 'I-ORG', 'I-LOC', 'I-MISC', 'I-PER', 'B-PER', 'B-LOC', 'O', 'B-MISC']
Number of Labels:  9
What the dataset looks like:


,Sentence#,word,pos,ner
0,sentence#0,Sao,NC,B-LOC
1,sentence#0,Paulo,VMI,I-LOC
2,sentence#0,(,Fpa,O
3,sentence#0,Brasil,NC,B-LOC
4,sentence#0,),Fpt,O
5,sentence#0,",",Fc,O
6,sentence#0,23,Z,O
7,sentence#0,may,NC,O
8,sentence#0,(,Fpa,O
9,sentence#0,EFECOM,NP,B-ORG


In [11]:
sentences_train[3]

[('Imagínense', 'VMM', 'O'),
 ('ustedes', 'PP', 'O'),
 ('que', 'CS', 'O'),
 ('entre', 'SP', 'O'),
 ('aquellos', 'DD', 'O'),
 ('españoles', 'NC', 'O'),
 (',', 'Fc', 'O'),
 ('que', 'PR', 'O'),
 ('fueron', 'VSI', 'O'),
 ('quienes', 'PR', 'O'),
 ('llevaron', 'VMI', 'O'),
 ('a', 'SP', 'O'),
 ('Europa', 'VMN', 'B-LOC'),
 ('esos', 'DD', 'O'),
 ('dones', 'NC', 'O'),
 ('americanos', 'AQ', 'O'),
 (',', 'Fc', 'O'),
 ('se', 'P0', 'O'),
 ('hubiera', 'VAS', 'O'),
 ('impuesto', 'VMP', 'O'),
 ('la', 'DA', 'O'),
 ('patriotería', 'NC', 'O'),
 ('gastronómica', 'AQ', 'O'),
 (':', 'Fd', 'O'),
 ('patatas', 'NC', 'O'),
 ('y', 'CC', 'O'),
 ('tomates', 'NC', 'O'),
 ('se', 'P0', 'O'),
 ('hubieran', 'VAS', 'O'),
 ('quedado', 'VMP', 'O'),
 ('en', 'SP', 'O'),
 ('curiosidades', 'NC', 'O'),
 ('botánicas', 'AQ', 'O'),
 ('.', 'Fp', 'O')]

In [21]:
sentences_test[28]

[('Ambos', 'DN', 'O'),
 ('dirigentes', 'NC', 'O'),
 ('han', 'VAI', 'O'),
 ('coincidido', 'VMP', 'O'),
 ('en', 'SP', 'O'),
 ('advertir', 'VMN', 'O'),
 ('que', 'CS', 'O'),
 ('la', 'DA', 'O'),
 ('ETA', 'NC', 'B-ORG'),
 ('no', 'RN', 'O'),
 ('está', 'VMI', 'O'),
 ('acabada', 'AQ', 'O'),
 ('y', 'CC', 'O'),
 ('que', 'PR', 'O'),
 ('puede', 'VMI', 'O'),
 ('seguir', 'VMN', 'O'),
 ('con', 'SP', 'O'),
 ('su', 'DP', 'O'),
 ('actuación', 'NC', 'O'),
 ('violenta', 'AQ', 'O'),
 ('durante', 'SP', 'O'),
 ('varios', 'DI', 'O'),
 ('años', 'NC', 'O'),
 ('si', 'CS', 'O'),
 ('no', 'RN', 'O'),
 ('se', 'P0', 'O'),
 ('articulan', 'VMI', 'O'),
 ('soluciones', 'NC', 'O'),
 ('políticas', 'AQ', 'O'),
 (',', 'Fc', 'O'),
 ('entre', 'SP', 'O'),
 ('las', 'DA', 'O'),
 ('que', 'CS', 'O'),
 ('Montero', 'NC', 'B-PER'),
 ('ha', 'VAI', 'O'),
 ('citado', 'VMP', 'O'),
 ('la', 'DA', 'O'),
 ('necesidad', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('que', 'PR', 'O'),
 ('se', 'P0', 'O'),
 ('dé', 'VMS', 'O'),
 ('a', 'SP', 'O'),
 ('las', 'DA'

# Extracción de features

Definimos las features que queremos usar.

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    # TODO

# WI - palabra actual
    # mayúsculas
    # sustantivo (pos tag)
    # stop word
    # signo de puntuacion + caracteres especiales
    # números
    # longitud
    
# W3 - palabra anterior
    # mayúsculas
    # sustantivo (pos tag)
    # stop word
    # signo de puntuacion + caracteres especiales
    # números
    # longitud
    
# W5 - palabra siguiente
    # mayúsculas
    # sustantivo (pos tag)
    # stop word
    # signo de puntuacion + caracteres especiales
    # números
    # longitud
    
    
    return features

In [ ]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
def sent2labels(sent):
    return [word[2] for word in sent]

# Datos para entrenamiento, validación y testeo

In [ ]:
X_train = [sent2features(s) for s in sentences_train]
y_train = [sent2labels(s) for s in sentences_train]

X_test = [sent2features(s) for s in sentences_test]
y_test = [sent2labels(s) for s in sentences_test]

X_val = [sent2features(s) for s in sentences_val]
y_val = [sent2labels(s) for s in sentences_val]

In [ ]:
X_train[0][0]

# Entrenamiento

In [ ]:
crf_ = sklearn_crfsuite.CRF(
    # TODO
)
crf_.fit(X_train, y_train)

# Evaluación

Emplear los datos de validación para _tunnear_ el modelo

In [ ]:
def eval(X, y, removeO=True):
    y_pred = crf_.predict(X)
    labels = list(crf_.classes_)
    
    if removeO:
        labels.remove('O')
    
    f1_score_ = metrics.flat_f1_score(y, y_pred, average='weighted', labels=labels)
    print('F1 score: {0:.3f}'.format(f1_score_))
    
    # Group B and I results
    sorted_labels = sorted(
        labels,
        key=lambda name: (name[1:], name[0])
    )
    
    classification_report_ = metrics.flat_classification_report(y, y_pred, labels=sorted_labels, digits=3)
    print(classification_report_)

In [ ]:
eval(X_val, y_val)

# Resultados finales

Una vez ajustado el modelo, usar el test set para evaluar el modelo final.

In [ ]:
eval(X_test, y_test)

# Transiciones más probables / improbables

In [ ]:
from collections import Counter

In [ ]:
def print_transitions(transition_features):
    for (label_from, label_to), weight in transition_features:
        print('{0:10}->   {1:10}->   {2:10}'.format(label_from, label_to, weight))

In [ ]:
common_transitions = Counter(crf_.transition_features_).most_common(10)

print('Transiciones más probables')
print_transitions(common_transitions)

In [ ]:
uncommon_transitions = Counter(crf_.transition_features_).most_common()[-10:]

print('Transiciones más improbables')
print_transitions(uncommon_transitions)

# Características estado

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print('{0:10}->   {1:10}->   {2:10}'.format(weight, label, attr))

In [ ]:
positive_state_features = Counter(crf_.state_features_).most_common(10)

print('Las más positivas')
print_state_features(positive_state_features)

In [ ]:
negative_state_features = Counter(crf_.state_features_).most_common()[-10:]

print('Las más negativas')
print_state_features(negative_state_features)